In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pathlib import Path
import pywt
from tqdm import tqdm

In [2]:
def find_episodes(arr):
    episodes = []
    start_index = None
    in_episode = False
    for i, val in enumerate(arr):
        if val != 0 and not in_episode:
            start_index = i
            in_episode = True
        elif val == 0 and in_episode:
            episodes.append([start_index, i - 1])
            in_episode = False
    # Check if the last element is part of an episode
    if in_episode:
        episodes.append([start_index, len(arr) - 1])
    return np.array(episodes)

In [3]:
def cropping(video_path, output_folder, episodes):
    import cv2 
    if Path(f'{output_folder}').exists():
        import shutil
        shutil.rmtree(f'{output_folder}')
    
    Path(output_folder).mkdir(parents=True, exist_ok = True)
    fps = 200
    frame_adjustment = int(fps * 1)  # 1 second worth of frames
    
    # Open the video
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Function to add text to a frame
    def add_text_to_frame(frame, text):
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_color = (255, 255, 255)  # White color
        font_thickness = 2
        cv2.putText(frame, text, (50, 50), font, font_scale, font_color, font_thickness)
        return frame
    
    # Iterate through the frame ranges
    for i, (start_frame, end_frame) in enumerate(episodes):
        # Adjust start and end frames
        adjusted_start = max(0, start_frame - frame_adjustment)
        adjusted_end = min(total_frames - 1, end_frame + frame_adjustment)
    
        # Set the start frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, adjusted_start)
    
        # Define the output video writer
        output_path = f'{output_folder}clip_{i}_{int(start_frame/fps)}-{int(end_frame/fps)}.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps * 0.2, (width, height))
    
        # Loop through the frames in the range
        for frame_num in range(adjusted_start, adjusted_end + 1):
            ret, frame = cap.read()
            if not ret:
                break
    
            # Add "episode" text to the frame
            if frame_num in list(range(start_frame, end_frame)):
                frame = add_text_to_frame(frame, "episode")
    
            # Write the frame to the output video
            out.write(frame)
    
        # Release the output video writer
        out.release()
        print(f"clips {i} is ready")
    
    # Release the video capture object
    cap.release()
    
    print("Clips have been successfully created.")

In [4]:
def wavelet_denoise(signal, wavelet='db4', level=3, threshold_type='soft'):
        coeffs = pywt.wavedec(signal, wavelet, mode='per', level=level)  # Decompose
        sigma = np.median(np.abs(coeffs[-level])) / 0.6745  # Noise estimation (MAD)
        threshold = sigma * np.sqrt(2 * np.log(len(signal)))  # Universal threshold
        coeffs_thresh = [pywt.threshold(c, threshold, mode=threshold_type) for c in coeffs]
        return pywt.waverec(coeffs_thresh, wavelet, mode='per')  # Reconstruct

In [5]:
def filtering(signal):
    def wavelet_denoise(signal, wavelet='db4', level=3, threshold_type='soft'):
        coeffs = pywt.wavedec(signal, wavelet, mode='per', level=level)  # Decompose
        sigma = np.median(np.abs(coeffs[-level])) / 0.6745  # Noise estimation (MAD)
        threshold = sigma * np.sqrt(2 * np.log(len(signal)))  # Universal threshold
        coeffs_thresh = [pywt.threshold(c, threshold, mode=threshold_type) for c in coeffs]
        return pywt.waverec(coeffs_thresh, wavelet, mode='per')  # Reconstruct
    
    def noise_detection(smoothed_signal, noise_window=10,
                             min_duration=3, prominence_factor=2.0):
        # 1. Estimate noise characteristics using rolling baseline
        noise_estimates = []
        for i in range(len(smoothed_signal)):
            start = max(0, i - noise_window)
            end = min(len(smoothed_signal), i + noise_window)
            noise_estimates.append(np.std(smoothed_signal[start:end]))
        
        noise_std = np.median(noise_estimates)  # Robust noise estimate
        return noise_std

    def be_zero(buffer, noise):
        signal = list(buffer)
        for i in range(len(signal)):
            if signal[i] < noise and signal[i] > -noise:
                signal[i] = 0
        return signal
    
    est = noise_detection(signal)
    filt =  wavelet_denoise(signal)
    return be_zero(filt, est)

In [6]:
folders = glob("F:\\swim_motion_data\\lcr_kai\\*")

for folder in tqdm(folders):

    profix = folder.split("\\")[-1]
    print(profix)
    save_dir = f'F:\\swim_motion_data\\lcr_kai_v3\\{profix}'
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    date = profix.split("_")[0]
    fish_no = profix.split("_")[1]
    set_no = profix.split("-")[2][3:]
    trial_no = profix.split("-")[-1][1:]
    
    h5_path = f"F:\\Temp_opto\\{date}\\lcr\\TOPCAMERA\\{fish_no}\\{set_no}\\{date}_{set_no}_Trial{trial_no}.h5"
    #### load top cam data
    top_df = pd.read_csv(f"{folder}\\TOPCAM\\y_shift_idx.csv")
    y_motions = np.load(f"{folder}\\TOPCAM\\y_shift.npy")
    up_idxs = top_df["ori"].values[top_df["is_up"].values == 1]
    down_idxs = top_df["ori"].values[top_df["is_up"].values == -1]
    
    #### load side cam data
    side_df = pd.read_csv(f"{folder}\\SIDECAM\\x_shift_idx.csv")
    x_motions = np.load(f"{folder}\\SIDECAM\\x_shift.npy")
    left_idxs = side_df["ori"].values[side_df["is_left"].values == 1]
    right_idxs = side_df["ori"].values[side_df["is_left"].values == -1]
    
    #### load old episode
    top_episodes = np.load(f"{folder}\\TOPCAM\\episodes_idx.npy")
    side_episodes = np.load(f"{folder}\\SIDECAM\\episodes_idx.npy")
    
    filt_y = filtering(y_motions)
    filt_x = filtering(x_motions)
    new_top_episodes = find_episodes(filt_y)
    new_side_episodes = find_episodes(filt_x)

    cropping(h5_path.split('.')[0]+".mp4",f"{save_dir}\\TOPCAM\\clips\\", new_top_episodes)
    sidecam_path = f"{h5_path.split("\\TO")[0]}\\SIDECAMERA\\{fish_no}\\{set_no}\\FLIR_{profix}-0000.avi"
    cropping(sidecam_path, f"{save_dir}\\SIDECAM\\clips\\", new_side_episodes)
    np.save(f"{save_dir}\\TOPCAM\\events.npy" , new_top_episodes)
    np.save(f"{save_dir}\\SIDECAM\\events.npy" ,new_top_episodes)

  0%|                                                                                           | 0/43 [00:00<?, ?it/s]

2024-08-01_F1_01-T2
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
Clips have been successfully created.
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20

  2%|█▉                                                                                 | 1/43 [01:05<45:57, 65.65s/it]

clips 93 is ready
Clips have been successfully created.
2024-08-01_F1_01-T3
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
clips 33 is ready
clips 34 is ready
clips 35 is ready
clips 36 is ready
clips 37 is ready
clips 38 is ready
clips 39 is ready
clips 40 is ready
clips 41 is ready
clips 42 is ready
clips 43 is ready
Clips have been successfully created.
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
cl

  5%|███▊                                                                             | 2/43 [02:51<1:01:07, 89.44s/it]

...

clips 42 is ready
Clips have been successfully created.
2024-08-29_F4_01-T8
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
Clips have been successfully created.
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready


 98%|████████████████████████████████████████████████████████████████████████████████  | 42/43 [40:34<00:32, 32.19s/it]

clips 28 is ready
Clips have been successfully created.
2024-08-29_F4_01-T9
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
clips 33 is ready
clips 34 is ready
clips 35 is ready
clips 36 is ready
clips 37 is ready
clips 38 is ready
Clips have been successfully created.
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [41:29<00:00, 57.90s/it]

clips 63 is ready
Clips have been successfully created.
